In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2

import warnings
warnings.filterwarnings("ignore")

import glob

from sklearn.model_selection import train_test_split

In [2]:
masks = glob.glob("../data/Dataset_BUSI_with_GT/*/*_mask.png")

In [6]:
masks[:4]

['../data/Dataset_BUSI_with_GT\\benign\\benign (1)_mask.png',
 '../data/Dataset_BUSI_with_GT\\benign\\benign (10)_mask.png',
 '../data/Dataset_BUSI_with_GT\\benign\\benign (100)_mask.png',
 '../data/Dataset_BUSI_with_GT\\benign\\benign (101)_mask.png']

In [4]:
images = [mask_images.replace("_mask", "") for mask_images in masks]

In [8]:
images[:4]

['../data/Dataset_BUSI_with_GT\\benign\\benign (1).png',
 '../data/Dataset_BUSI_with_GT\\benign\\benign (10).png',
 '../data/Dataset_BUSI_with_GT\\benign\\benign (100).png',
 '../data/Dataset_BUSI_with_GT\\benign\\benign (101).png']

In [9]:
series = list(zip(images, masks))

In [10]:
series[:4]

[('../data/Dataset_BUSI_with_GT\\benign\\benign (1).png',
  '../data/Dataset_BUSI_with_GT\\benign\\benign (1)_mask.png'),
 ('../data/Dataset_BUSI_with_GT\\benign\\benign (10).png',
  '../data/Dataset_BUSI_with_GT\\benign\\benign (10)_mask.png'),
 ('../data/Dataset_BUSI_with_GT\\benign\\benign (100).png',
  '../data/Dataset_BUSI_with_GT\\benign\\benign (100)_mask.png'),
 ('../data/Dataset_BUSI_with_GT\\benign\\benign (101).png',
  '../data/Dataset_BUSI_with_GT\\benign\\benign (101)_mask.png')]

In [11]:
dataset = pd.DataFrame(series, columns=['image_path', 'mask_path'])

In [12]:
dataset

,image_path,mask_path
0,../data/Dataset_BUSI_with_GT\benign\benign (1)...,../data/Dataset_BUSI_with_GT\benign\benign (1)...
1,../data/Dataset_BUSI_with_GT\benign\benign (10...,../data/Dataset_BUSI_with_GT\benign\benign (10...
2,../data/Dataset_BUSI_with_GT\benign\benign (10...,../data/Dataset_BUSI_with_GT\benign\benign (10...
3,../data/Dataset_BUSI_with_GT\benign\benign (10...,../data/Dataset_BUSI_with_GT\benign\benign (10...
4,../data/Dataset_BUSI_with_GT\benign\benign (10...,../data/Dataset_BUSI_with_GT\benign\benign (10...
...,...,...
775,../data/Dataset_BUSI_with_GT\normal\normal (95...,../data/Dataset_BUSI_with_GT\normal\normal (95...
776,../data/Dataset_BUSI_with_GT\normal\normal (96...,../data/Dataset_BUSI_with_GT\normal\normal (96...
777,../data/Dataset_BUSI_with_GT\normal\normal (97...,../data/Dataset_BUSI_with_GT\normal\normal (97...
778,../data/Dataset_BUSI_with_GT\normal\normal (98...,../data/Dataset_BUSI_with_GT\normal\normal (98...


In [13]:
train, test= train_test_split(dataset, test_size=0.2)

In [19]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
torch.cuda.set_device(0)

In [64]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import pandas as pd
import cv2 as cv

class CustomImageMaskDataset(Dataset):
    def __init__(self, dataframe, image_transform=None):
        self.data = dataframe
        self.image_transform = image_transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx, 0]
        mask_path = self.data.iloc[idx, 1]

        image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
        mask = cv.imread(mask_path, cv.IMREAD_GRAYSCALE)

        if self.image_transform:
            image = self.image_transform(image)

        return image, mask

# Define transformations for images and masks
train_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

val_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

In [65]:
custom_dataset = CustomImageMaskDataset(train, train_transforms)

# Create a data loader to iterate over the dataset
batch_size = 32  # Set your desired batch size
dataloader = torch.utils.data.DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)


In [66]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [67]:
device

device(type='cuda', index=0)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class ConvBlock(nn.Module):

    def __init__(self, input_channel, out_channel):
        super().__init__()
        self.conv2d_1 = nn.Conv2d(input_channel, out_channel, kernel_size=3, padding='same')
        self.batchnorm_1 = nn.BatchNorm2d(out_channel)
        self.gelu_1 = nn.GELU()

        self.conv2d_2 = nn.Conv2d(input_channel, out_channel, kernel_size=3, padding='same')
        self.batchnorm_2 = nn.BatchNorm2d(out_channel)
        self.gelu_2 = nn.GELU()

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.batchnorm_1(x)
        x = self.gelu_1(x)

        x = self.conv2d_2(x)
        x = self.batchnorm_2(x)
        x = self.gelu_2(x)

        return x

class Encoder(nn.Module):

    def __init__(self, input_channel, out_channel):
        super().__init__()
        self.conv2d_1 = ConvBlock(input_channel, out_channel)
        self.maxpool = nn.MaxPool2d((2,2))

    def forward(self, x):
        x = self.conv2d_1(x)
        p = self.maxpool(x)

        return x, p

class Decoder(nn.Module):

    def __init__(self, input_channel, output_channel):
        super().__init__()
        self.conv_t = nn.ConvTranspose2d(input_channel, output_channel, stride=2, kernel_size=3)
        self.conv2d_1 = ConvBlock(input_channel, output_channel)

    def forward(self, x, skip):
        x = self.conv_t(x)
        x = torch.cat([x, skip], dim=1)
        x = self.conv2d_1(x)

        return x

In [3]:
class Unet(nn.Module):

    def __init__(self, input_chennel=3):
        super().__init__()
        self.encoder_1 = Encoder(input_chennel, 64)
        self.encoder_2 = Encoder(64, 128)
        self.encoder_3 = Encoder(128, 256)
        self.encoder_4 = Encoder(256, 512)

        self.conv_block = ConvBlock(512, 1024)

        self.decoder_1 = Decoder(1024, 512)
        self.decoder_2 = Decoder(512, 256)
        self.decoder_3 = Decoder(256, 128)
        self.decoder_4 = Decoder(128, 64)

        self.cls = nn.Conv2d(64, 1, kernel_size=1, padding=0)

    def forward(self, x):

        """ ------ Encoder ------"""
        x1, p1 = self.encoder_1(x)
        x2, p2 = self.encoder_2(p1)
        x3, p3 = self.encoder_3(p2)
        x4, p4 = self.encoder_4(p3)

        """ ------ BottleNeck ------"""
        x6 = self.conv_block(p4)

        """ ------ Decoder ------"""
        x7 = self.decoder_1(x6, x4)
        x8 = self.decoder_1(x7, x3)
        x9 = self.decoder_1(x8, x2)
        x10 = self.decoder_1(x9, x1)

        """ ------ Final Layer ------"""
        x_final = self.cls(x10)

        return x_final
